In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()


In [ ]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

In [ ]:
def chat(message,history):
  history = [{'role':h['user'],'content':h['content']} for h in history]
  messages = [{'role':'system','content':system_message}]+history+[{'role':'user','content':message}]
  response = openai.chat.completions.create(model=MODEL,messages=messages)

  return response.choices[0].message.content
  


TOOLS

In [ ]:

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [ ]:
get_ticket_price("INDIA")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "transit_points":{
                "type":'string',
                "description":'The intermediate stop point favoured by the user'
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type":"function","function":price_function}]

In [ ]:
def handle_tool_call(message):
  tool_call=message.tool_calls[0]
  if tool_call.function.name == 'get_ticket_price':
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price_details = get_ticket_price(city)
    response = {
      "role":'tool',
      "content":price_details,
      "tool_call_id":tool_call.id
    }
  return response

In [ ]:
def chat(message,history):
  history = [{"role":h['role'],"content":h['content']} for h in history]
  messages = [{"role":'system','content':system_message}]+history+[{"role": "user", "content": message}]
  response = openai.chat.completions.create(model=MODEL,messages=messages,tools=tools)

  if response.choices[0].finish_reason =="tool_calls":
    message = response.choices[0].message
    response = handle_tool_call(message)
    messages.append(message)
    messages.append(response)
    response = openai.chat.completions.create(model=MODEL,messages=messages)
  return response.choices[0].message.content


In [ ]:
def handle_tool_call(message):
  tool_call =message.tool_calls[0]
  if tool_call.function.name == "get_ticket_price":
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price_details = get_ticket_price(city)
    response = {
      "role":'tool',
      "content":price_details,
      'tool_call_id':tool_call.id
    }

Train Availbility

In [ ]:
system_message = [{'role':'system','content':"You are an helpful assistant that checks the train availablity give short answers no more than 1 sentences. You can use a tool called check_availability to check for availablity"}]

In [ ]:
def check_availability(destination_city):
  train_content = {'madurai':True,'chennai':True}

  if train_content.get(destination_city.lower()):
    return f"Ticket is available for {destination_city}"
  else:
    return "No Ticket is not available at the moment"
  
  

In [ ]:
def handle_tool_call(message):
  responses = []
  for tool_call in message.tool_calls:
    if tool_call.function.name == "get_ticket_price":
      arguments = json.loads(tool_call.function.arguments)
      city = arguments.get('destination_city')
      print(message)
      price_details = get_ticket_price(city)
      responses.append( {
        'role':'tool',
        'content':price_details,
        'tool_call_id':tool_call.id
      })
  return responses

In [ ]:
## Now we need to let know the model a tool is there for it to use
check_function = {
  "name":'get_ticket_price',
  "description":'it gets whether the train is available or not depending on the city name which is passed',
  "parameters":{
    "type":"object",
    'properties':{
      "destination_city":{
      "type":'string',
      "description":"The city that the customer wishes to travel to"
    }
    },
    "required":["destination_city"],
    "additionalProperties":False    
  }
}

In [ ]:
tools = [{"type":'function','function':check_function}]

In [ ]:
def chat(message,history):
  history=[{"role":h['role'],'content':h['content']} for h in history] 
  messages = system_message+history+[{'role':'user','content':message}]
  response = openai.chat.completions.create(model=MODEL,messages=messages,tools=tools)

  while response.choices[0].finish_reason=="tool_calls":
    message=response.choices[0].message
    response = handle_tool_call(message)
    messages.append(message)
    messages.extend(response)
    response = openai.chat.completions.create(model=MODEL, messages=messages,tools=tools)
  return response.choices[0].message.content



In [ ]:
gr.ChatInterface(fn=chat).launch()

In [ ]:
import sqlite3

In [ ]:
DB = "checklist.db"

with sqlite3.connect(DB) as conn:
  cursor =conn.cursor()
  cursor.execute('DROP TABLE IF EXISTS checklist')
  cursor.execute('CREATE TABLE checklist (city TEXT PRIMARY KEY, Available BOOLEAN)')
  conn.commit()

In [ ]:
def get_ticket_price(city):
  with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    (cursor.execute('SELECT Available FROM checklist WHERE city = ?', (city.lower(),)))
    result = cursor.fetchone()
    return f"Ticket price to {city} is ${result[0]}" if result else f"No price data available {city}"

get_ticket_price("London")

In [ ]:
import sqlite3

DB = "checklist.db"

# with sqlite3.connect(DB) as conn:
#     cursor = conn.cursor()
#     cursor.execute('CREATE TABLE availability (city TEXT PRIMARY KEY, Available BOOLEAN)')
#     conn.commit()

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("INSERT OR REPLACE INTO availability (city, Available) VALUES (?, ?)", ("london", True))
    conn.commit()

def get_ticket_price(city):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT Available FROM availability WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is available" if result else f"No ticket is not available for {city}"

print(get_ticket_price("Paris"))


In [ ]:
gr.ChatInterface(fn=chat).launch()

Now lets see